## test9.ipynb

Compare a ei_network simulation against an equivalent ei_net simulation.

Two simulations are compared. In the first one, an 8x8 grid of E and I units is created using ei_net.
In the second one, the same grid is created as four 4x4 ei_network layers, each covering a "quadrant" of the original grid.

By Sergio Verduzco
January 2018

In [1]:
%cd ..
#import ei_network
from ei_network import *
from ei_net import *

/home/z/projects/draculab


In [2]:
# Create the ei_net network
net0 = ei_net()

In [3]:
# Set the ei_net parameters
# We homogeneize the connections
for conn in ['ee_conn', 'ei_conn', 'ie_conn', 'ii_conn', 'xe_conn', 'xi_conn']:
    net0.set_param(conn, 'edge_wrap', False)
    net0.set_param(conn, 'mask',  {'circular': {'radius': .3}})
    net0.set_param(conn, 'kernel',  1.)
for conn in ['xe_conn', 'xi_conn']:
    net0.set_param(conn, 'weights', {'linear' : {'c' : 2.5, 'a' : 2.}})
for conn in ['ee_conn', 'ei_conn']:
    net0.set_param(conn, 'weights', {'linear' : {'c' : .5, 'a' : 1.}})
for conn in ['ie_conn', 'ii_conn']:
    net0.set_param(conn, 'weights', {'linear' : {'c' : -.5, 'a' : 1.}})
# Same geometry as the other network
for geom in ['e_geom', 'i_geom', 'x_geom']:
    net0.set_param(geom, 'center', [0., 0.])
    net0.set_param(geom, 'extent', [2., 2.])
    net0.set_param(geom, 'rows', 8)
    net0.set_param(geom, 'columns', 8)
    net0.set_param(geom, 'jitter', 0.)
net0.set_param('x_geom', 'rows', 2)
net0.set_param('x_geom', 'columns', 2)
net0.n['w_track'] = 8

In [4]:
# Build the ei_net network
net0.build()

In [5]:
# Create ei_network network
layer_names = ['LD', 'LU', 'RD', 'RU'] # left down, left up, right down, right up
net1 = ei_network(layer_names)

In [6]:
# Configure layer parameters
LD = net1.layers['LD']
LU = net1.layers['LU']
RD = net1.layers['RD']
RU = net1.layers['RU']

centers = ( [-0.5,-0.5], [-0.5, 0.5], [0.5,-0.5], [0.5, 0.5] )
layers = [LD, LU, RD, RU]
for lyr, cntr in zip(layers, centers):
    for geom in ['e_geom', 'i_geom', 'x_geom']:
        lyr.set_param(geom, 'center', cntr)
        lyr.set_param(geom, 'extent', [1., 1.])
        lyr.set_param(geom, 'jitter', 0.)
        lyr.set_param(geom, 'rows', 4)
        lyr.set_param(geom, 'columns', 4)
    # A single input unit per layer, located at the center of the layer
    lyr.set_param('x_geom', 'rows', 1)
    lyr.set_param('x_geom', 'columns', 1)
    # match the connection parameters
    lyr.set_param('ee_conn', 'weights', {'linear' : {'c' : .5, 'a' : 1.}})
    lyr.set_param('ei_conn', 'weights', {'linear' : {'c' : .5, 'a' : 1.}})
    lyr.set_param('ie_conn', 'weights', {'linear' : {'c' : -.5, 'a' : 1.}})
    lyr.set_param('ii_conn', 'weights', {'linear' : {'c' : -.5, 'a' : 1.}})
    lyr.set_param('xe_conn', 'weights', {'linear' : {'c' : 2.5, 'a' : 2.}})
    lyr.set_param('xi_conn', 'weights', {'linear' : {'c' : 2.5, 'a' : 2.}})
    for conn in ['ee_conn', 'ei_conn', 'ie_conn', 'ii_conn', 'xe_conn', 'xi_conn']:
        lyr.set_param(conn, 'edge_wrap', False)
        lyr.set_param(conn, 'mask', {'circular': {'radius': .3}})
        lyr.set_param(conn, 'kernel', 1.)
    lyr.n['w_track'] = 2
    

In [7]:
# Add interlayer connections
pops = ['e', 'i']
# We'll allow any two populations to connect
for source in layer_names:
    for target in layer_names:
        if source != target:
            for pop1 in pops:
                for pop2 in pops:
                    net1.add_connection((source, pop1), (target, pop2))

In [8]:
# Configure interlayer connections
# I don't need to setup any interlayer transformations, just avoid periodic boundaries
for source in layer_names:
    for target in layer_names:
        if source != target:
            for pop1 in pops:
                for pop2 in pops:
                    net1.set_param(source+pop1+'_'+target+pop2+'_conn', 'edge_wrap', False )
                    net1.set_param(source+pop1+'_'+target+pop2+'_conn', 'mask', {'circular': {'radius': .3}})
                    net1.set_param(source+pop1+'_'+target+pop2+'_conn', 'kernel', .9)
                    net1.set_param(source+pop1+'_'+target+pop2+'_conn', 'transform', None)
                    if pop1 == 'e':
                        net1.set_param(source+pop1+'_'+target+pop2+'_conn', 'weights',  {'linear' : {'c' : .5, 'a' : 1.}})
                    else:
                        net1.set_param(source+pop1+'_'+target+pop2+'_conn', 'weights',  {'linear' : {'c' : -.5, 'a' : 1.}})

In [9]:
net1.build()

Building layer LU
Building layer RD
Building layer RU
Building layer LD
Creating LDe_LUe connection
Creating LDe_LUi connection
Creating LDi_LUe connection
Creating LDi_LUi connection
Creating LDe_RDe connection
Creating LDe_RDi connection
Creating LDi_RDe connection
Creating LDi_RDi connection
Creating LDe_RUe connection
Creating LDe_RUi connection
Creating LDi_RUe connection
Creating LDi_RUi connection
Creating LUe_LDe connection
Creating LUe_LDi connection
Creating LUi_LDe connection
Creating LUi_LDi connection
Creating LUe_RDe connection
Creating LUe_RDi connection
Creating LUi_RDe connection
Creating LUi_RDi connection
Creating LUe_RUe connection
Creating LUe_RUi connection
Creating LUi_RUe connection
Creating LUi_RUi connection
Creating RDe_LDe connection
Creating RDe_LDi connection
Creating RDi_LDe connection
Creating RDi_LDi connection
Creating RDe_LUe connection
Creating RDe_LUi connection
Creating RDi_LUe connection
Creating RDi_LUi connection
Creating RDe_RUe connection
Crea

/home/z/projects/draculab/topology.py:390: UserWarning: Zero connections created with topo_connect
  warn('Zero connections created with topo_connect', UserWarning)


In [10]:
# counting units
nu_n0 = len(net0.net.units)
nu_n1 = len(net1.net.units)
print("UNITS - net0: %d, net1: %d " % (nu_n0, nu_n1))
# counting synapses
synsum0 = 0
for slist in net0.net.syns:
    synsum0 += len(slist)
synsum1 = 0
for slist in net1.net.syns:
    synsum1 += len(slist)
print("SYNAPSES - net0 : %d, net1 : %d" % (synsum0, synsum1))

UNITS - net0: 140, net1: 140 
SYNAPSES - net0 : 1184, net1 : 1177


In [17]:
net1.layers['LU'].n['w_track']

6

In [ ]:
ei_net.run(5, 1.)

In [ ]:
%matplotlib inline
ei_net.basic_plot('S')

In [ ]:
ei_net.basic_plot('M')

In [ ]:
ei_net.double_anim(M.e+S.e+S.x, thr=0.45)

In [ ]:
# Plot the highest unit activities
%matplotlib inline
n_top = 4  # how many units to plot
top_pop = np.array(S.e)  # population to visualize
top_units = ei_net.all_activs[top_pop].sum(1).argpartition(-n_top)[-n_top:]
print(top_units)
top_act_fig = plt.figure(figsize=(18,8))
top_acts = ei_net.all_activs[top_pop[top_units]].transpose()
plt.plot(ei_net.all_times, top_acts)
plt.show()

In [ ]:
# Plot the lowest unit activities
n_bottom = 4  # how many units to plot
bottom_pop = np.array(M.e)  # population to visualize
bottom_units = ei_net.all_activs[bottom_pop].sum(1).argpartition(n_bottom)[0:n_bottom-1]
print(bottom_units)
bottom_act_fig = plt.figure(figsize=(18,8))
bottom_acts = ei_net.all_activs[bottom_pop[bottom_units]].transpose()
plt.plot(ei_net.all_times, bottom_acts)
plt.show()

In [11]:
e = LD.e+LU.e+RD.e+RU.e 
x = LD.x+LU.x+RD.x+RU.x
i = LD.i+LU.i+RD.i+RU.i
net1.conn_anim(e, e, slider=True, weights=True)

interactive(children=(IntSlider(value=32, description='frame', max=64, min=1), Output()), _dom_classes=('widget-interact',))

<function ipywidgets.widgets.interaction._InteractFactory.__call__.<locals>.<lambda>>

In [13]:
net0.conn_anim(net0.e+net0.x, net0.e, slider=True, weights=True)

interactive(children=(IntSlider(value=34, description='frame', max=68, min=1), Output()), _dom_classes=('widget-interact',))

<function ipywidgets.widgets.interaction._InteractFactory.__call__.<locals>.<lambda>>

In [16]:
sou_u = 118
targets = net1.id2snk[ [syn.postID for syn in net1.all_syns if syn.preID == sou_u ] ] 
targets

array([], dtype=int64)

In [21]:
sou_u = 118
sink_0 = RD.e[0]
source_0 = RD.e[0]
targets = [syn.postID - sink_0 for syn in net1.all_syns if syn.preID == sou_u + source_0] 
targets

[]

In [ ]:
ei_net.act_anim(S.e+M.e, 0.2)

In [ ]:
ei_net.hist_anim(S.e+M.e)

In [ ]:
# logging
ei_net.log()

In [ ]:
a = np.tile(1e6,(4,1))
a[2] = 1
a[ [0, 2, 3]]